# Exploratory Data Analysis (EDA)

This notebook performs comprehensive exploratory data analysis on the Aadhaar enrollment and update data.

## Objectives:
1. Understand data distributions
2. Identify patterns and trends
3. Analyze state-wise and temporal variations
4. Detect correlations between variables

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print("✓ Libraries imported successfully")

## 1. Load Cleaned Data

In [ ]:
# Load the unified analytical table
df = pd.read_csv('../data/processed/unified_analytical_table.csv')
df['date'] = pd.to_datetime(df['date'])

print(f"Data shape: {df.shape}")
print(f"\nDate range: {df['date'].min()} to {df['date'].max()}")
print(f"\nNumber of states: {df['state'].nunique()}")
print(f"Number of districts: {df['district'].nunique()}")

df.head()

## 2. Statistical Summary

In [ ]:
# Descriptive statistics
df.describe()

## 3. Distribution Analysis

In [ ]:
# Distribution of total enrollments
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Enrollment distribution
axes[0, 0].hist(df['total_enrolment'], bins=50, edgecolor='black')
axes[0, 0].set_title('Distribution of Total Enrollments', fontsize=12, fontweight='bold')
axes[0, 0].set_xlabel('Total Enrollments')
axes[0, 0].set_ylabel('Frequency')

# Update distribution
axes[0, 1].hist(df['total_updates'], bins=50, edgecolor='black', color='orange')
axes[0, 1].set_title('Distribution of Total Updates', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('Total Updates')
axes[0, 1].set_ylabel('Frequency')

# Update Pressure Index
axes[1, 0].hist(df['update_pressure_index'], bins=50, edgecolor='black', color='green')
axes[1, 0].set_title('Distribution of Update Pressure Index', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('Update Pressure Index')
axes[1, 0].set_ylabel('Frequency')

# Demo vs Bio updates
axes[1, 1].scatter(df['total_demo_updates'], df['total_bio_updates'], alpha=0.5)
axes[1, 1].set_title('Demographic vs Biometric Updates', fontsize=12, fontweight='bold')
axes[1, 1].set_xlabel('Demographic Updates')
axes[1, 1].set_ylabel('Biometric Updates')

plt.tight_layout()
plt.show()

## 4. Temporal Trends

In [ ]:
# Monthly trends
monthly_data = df.groupby(df['date'].dt.to_period('M')).agg({
    'total_enrolment': 'sum',
    'total_updates': 'sum',
    'total_demo_updates': 'sum',
    'total_bio_updates': 'sum'
}).reset_index()

monthly_data['date'] = monthly_data['date'].dt.to_timestamp()

fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Enrollment Trends Over Time', 'Update Trends Over Time'),
    vertical_spacing=0.15
)

# Enrollments
fig.add_trace(
    go.Scatter(x=monthly_data['date'], y=monthly_data['total_enrolment'],
               mode='lines+markers', name='Total Enrollments',
               line=dict(color='blue', width=2)),
    row=1, col=1
)

# Updates
fig.add_trace(
    go.Scatter(x=monthly_data['date'], y=monthly_data['total_demo_updates'],
               mode='lines+markers', name='Demographic Updates',
               line=dict(color='orange', width=2)),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=monthly_data['date'], y=monthly_data['total_bio_updates'],
               mode='lines+markers', name='Biometric Updates',
               line=dict(color='green', width=2)),
    row=2, col=1
)

fig.update_layout(height=800, title_text="Temporal Trends Analysis", showlegend=True)
fig.show()

## 5. State-wise Analysis

In [ ]:
# Top 10 states by enrollment
state_summary = df.groupby('state').agg({
    'total_enrolment': 'sum',
    'total_updates': 'sum',
    'update_pressure_index': 'mean'
}).reset_index()

state_summary = state_summary.sort_values('total_enrolment', ascending=False)

fig = px.bar(
    state_summary.head(10),
    x='state',
    y='total_enrolment',
    title='Top 10 States by Total Enrollments',
    labels={'total_enrolment': 'Total Enrollments', 'state': 'State'},
    color='total_enrolment',
    color_continuous_scale='Blues'
)

fig.update_layout(xaxis_tickangle=-45, height=500)
fig.show()

## 6. Correlation Analysis

In [ ]:
# Correlation matrix
numeric_cols = df.select_dtypes(include=[np.number]).columns
correlation_matrix = df[numeric_cols].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm',
            center=0, square=True, linewidths=1)
plt.title('Correlation Matrix of Aadhaar Metrics', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. Seasonality Detection

In [ ]:
# Monthly patterns (boxplot)
df['month'] = df['date'].dt.month
df['month_name'] = df['date'].dt.month_name()

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Enrollment seasonality
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

sns.boxplot(data=df, x='month_name', y='total_enrolment', order=month_order, ax=axes[0])
axes[0].set_title('Enrollment Seasonality by Month', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Month')
axes[0].set_ylabel('Total Enrollments')
axes[0].tick_params(axis='x', rotation=45)

# Update seasonality
sns.boxplot(data=df, x='month_name', y='total_updates', order=month_order, ax=axes[1])
axes[1].set_title('Update Seasonality by Month', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Month')
axes[1].set_ylabel('Total Updates')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print("\n📊 Seasonality Analysis:")
print("Seasonality is mild but observable in monthly patterns.")
print("This justifies using SARIMA over simple ARIMA for forecasting.")

## 8. Key Insights

### Summary of Findings:

1. **Temporal Patterns**: Clear monthly trends observed in both enrollments and updates
2. **State Variations**: Significant differences across states in enrollment and update volumes
3. **Seasonality**: Mild but observable seasonal patterns justify SARIMA modeling
4. **Correlations**: Strong positive correlation between demographic and biometric updates
5. **Distribution**: Right-skewed distributions suggest presence of high-demand districts

### Recommendations:
- Use SARIMA for forecasting (captures seasonal patterns)
- Focus resources on high-demand states and months
- Monitor Update Pressure Index for capacity planning

In [ ]:
print("✅ EDA Complete!")
print("\nNext steps: Proceed to 03_indicators.ipynb for derived indicator analysis")